In [ ]:
from __future__ import print_function
import keras
import tensorflow as tf
from tensorflow.keras import layers, Model
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.utils import to_categorical
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
from tensorflow.keras.models import load_model
from keras.applications import MobileNet
import os
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt


from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import math
import os
import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import zipfile
import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
# Importing libraries
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Embedding
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from google.colab import drive
import pandas as pd
import tensorflow_datasets as tfds
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.python.keras.layers import Input, Dense
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from sklearn import preprocessing

batch_size = 16
epochs = 250
data_augmentation = True
num_classes = 10
from keras.applications.vgg19 import VGG19
subtract_pixel_mean = True

In [ ]:
import os, re, time, json
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds

print("Tensorflow version " + tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Tensorflow version 2.9.2


In [ ]:
# BATCH_SIZE = 32 
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
(training_images, training_labels) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 52s 0us/step


In [ ]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_ims

In [ ]:
train_X = preprocess_image_input(training_images)
valid_X = preprocess_image_input(validation_images)

In [ ]:
def feature_extractor(inputs):

  feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor

def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x

In [ ]:
def final_model(inputs):

    resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)

    resnet_feature_extractor = feature_extractor(resize)
    classification_output = classifier(resnet_feature_extractor)

    return classification_output

def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model

In [ ]:
model = define_compile_model()

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d_1 (UpSampling  (None, 224, 224, 3)      0         
 2D)                                                             
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 1024)              2098

In [ ]:
def lr_schedule(epoch):
  lr = 1e-3
  if epoch > 180:
    lr *= 0.5e-3
  elif epoch > 160:
    lr *= 1e-3
  elif epoch > 120:
    lr *= 1e-2
  elif epoch > 80:
    lr *= 1e-1
  print('Learning rate: ', lr)
  return lr


In [ ]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
  os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)
lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]


In [ ]:
# EPOCHS = 3
history = model.fit(train_X[:10000], training_labels[:10000], epochs=250, 
                    validation_data = (valid_X, validation_labels), batch_size=16, verbose=1, workers=4,
                    callbacks=callbacks)

Learning rate:  0.001
Epoch 1/250
625/625 [==============================] - ETA: 0s - loss: 1.5433 - accuracy: 0.5249

625/625 [==============================] - 148s 231ms/step - loss: 1.5433 - accuracy: 0.5249 - val_loss: 0.8257 - val_accuracy: 0.7473 - lr: 0.0010
Learning rate:  0.001
Epoch 2/250
625/625 [==============================] - ETA: 0s - loss: 0.6672 - accuracy: 0.7960

625/625 [==============================] - 144s 230ms/step - loss: 0.6672 - accuracy: 0.7960 - val_loss: 0.4835 - val_accuracy: 0.8398 - lr: 0.0010
Learning rate:  0.001
Epoch 3/250
625/625 [==============================] - ETA: 0s - loss: 0.4233 - accuracy: 0.8675

625/625 [==============================] - 143s 230ms/step - loss: 0.4233 - accuracy: 0.8675 - val_loss: 0.3772 - val_accuracy: 0.8727 - lr: 0.0010
Learning rate:  0.001
Epoch 4/250
625/625 [==============================] - ETA: 0s - loss: 0.2955 - accuracy: 0.9072

625/625 [==============================] - 131s 209ms/step - loss: 0.2955 - accuracy: 0.9072 - val_loss: 0.3228 - val_accuracy: 0.8910 - lr: 0.0010
Learning rate:  0.001
Epoch 5/250
625/625 [==============================] - ETA: 0s - loss: 0.2152 - accuracy: 0.9329

...

625/625 [==============================] - 143s 229ms/step - loss: 0.0044 - accuracy: 0.9993 - val_loss: 0.3272 - val_accuracy: 0.9178 - lr: 3.1623e-04
Learning rate:  0.001
Epoch 48/250
625/625 [==============================] - ETA: 0s - loss: 0.0040 - accuracy: 0.9996

625/625 [==============================] - 143s 229ms/step - loss: 0.0040 - accuracy: 0.9996 - val_loss: 0.3304 - val_accuracy: 0.9163 - lr: 0.0010
Learning rate:  0.001
Epoch 49/250
625/625 [==============================] - ETA: 0s - loss: 0.0036 - accuracy: 0.9996

625/625 [==============================] - 131s 209ms/step - loss: 0.0036 - accuracy: 0.9996 - val_loss: 0.3273 - val_accuracy: 0.9192 - lr: 0.0010
Learning rate:  0.001
Epoch 50/250
625/625 [==============================] - ETA: 0s - loss: 0.0044 - accuracy: 0.9995

625/625 [==============================] - 143s 228ms/step - loss: 0.0044 - accuracy: 0.9995 - val_loss: 0.3264 - val_accuracy: 0.9186 - lr: 0.0010
Learning rate:  0.001
Epoch 51/250
 40/625 [>.............................] - ETA: 1:34 - loss: 0.0034 - accuracy: 1.0000

KeyboardInterrupt: ignored

In [ ]:
loss, accuracy = model.evaluate(valid_X, validation_labels, batch_size=16)


625/625 [==============================] - 30s 47ms/step - loss: 0.3271 - accuracy: 0.9193


In [ ]:
model.save("resnet_50_best_till_now")

In [ ]:
!zip -r /content/resnet_50_best_till_now /content/resnet_50_best_till_now
# files.download("/content/resnet_50_best_till_now.zip")


  adding: content/resnet_50_best_till_now/ (stored 0%)
  adding: content/resnet_50_best_till_now/saved_model.pb (deflated 92%)
  adding: content/resnet_50_best_till_now/variables/ (stored 0%)
  adding: content/resnet_50_best_till_now/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/resnet_50_best_till_now/variables/variables.index (deflated 77%)
  adding: content/resnet_50_best_till_now/keras_metadata.pb (deflated 96%)
  adding: content/resnet_50_best_till_now/assets/ (stored 0%)


NameError: ignored

In [ ]:
model=load_model("/content/resnet_50_best_till_now")


In [ ]:
class Sampling:
  
  def __init__(self, samplingtype, dataset, model, goal, jump, first, epochs, type, X_train=train_X, y_train=training_labels, X_test=valid_X, y_test=validation_labels):
  
    self.samplingtype = samplingtype
    # self.model = model
    self.dataset = dataset
    self.goal = goal
    self.epochs=epochs
    self.jump = jump
    self.type=type
    # self.resnet = resnet
    self.first = first
    # if()
    self.model=model


    if(self.type=="resnet"):
      self.X_train = train_X
      self.y_train = training_labels
      self.X_test = valid_X
      self.y_test = validation_labels
      # y_train = to_categorical(y_train,num_classes=10)
      # y_test = to_categorical(y_test,num_classes=10)  

    else:
      if(dataset=="cifar10"):
        (self.X_train, self.y_train) , (self.X_test, self.y_test) = keras.datasets.cifar10.load_data()
        print(self.X_train.shape)
        print(self.y_train.shape)

      elif(dataset=="mnist"):
        (self.X_train, self.y_train) , (self.X_test, self.y_test) = keras.datasets.mnist.load_data()
      elif(dataset=="imdb"):
        (self.X_train, self.y_train), (self.X_test, self.y_test) = keras.datasets.imdb.load_data(num_words=30000)
      else:
        print("Input error!")

    if(samplingtype=="random"):
      random_num = np.random.randint(self.first,self.X_train.shape[0],size=self.X_train.shape[0]-self.first)
      self.random_num= random_num


  def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
      lr *= 0.5e-3
    elif epoch > 160:
      lr *= 1e-3
    elif epoch > 120:
      lr *= 1e-2
    elif epoch > 80:
      lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

  def get_iterations_rs(self):
    
    e=[0,0]
    
    k=0
    
    num=0
    
    y=[]
    x=[]
    for i in range(0,self.first):
      y.append(self.y_train[i])
      x.append(self.X_train[i])
    
    while((e[1]<self.goal/100)&(k+self.jump<self.X_train.shape[0])):

        for i in range(0+k,self.jump+k):
            x.append(self.X_train[self.random_num[i]])
            y.append(self.y_train[self.random_num[i]])
         
        k=k+self.jump
        num=num+1
        
        self.model.fit(np.array(x), np.array(y), epochs=250)
        
        e = self.model.evaluate(self.X_test, self.y_test)
    
    return e[1],num


  def get_entropy(self,y_predicted_en):
    sum_prob = 0
    entropy = []

    for i in range(0,y_predicted_en.shape[0]):
        for j in range(0,y_predicted_en.shape[1]):
          if(y_predicted_en[i][j]==0):
            continue
          k = y_predicted_en[i][j]*math.log(y_predicted_en[i][j])
          sum_prob = sum_prob+k
        entropy.append([int(i),-(sum_prob)])
        sum_prob=0
        k=0
    
    entropy.sort(key=lambda x:x[1],reverse=True)
    
    return entropy

  def get_lc(self,y_predicted_lc):
    probarray = []
    
    for i in range(0,y_predicted_lc.shape[0]):
        probarray.append([int(i),np.max(y_predicted_lc[i])])

    probarray.sort(key=lambda x:x[1])
    
    return probarray

  def get_ratio(self,y_predicted_r):
    ratio_array = []
    
    for i in range(0,y_predicted_r.shape[0]):
        y=y_predicted_r[i]
        sorted(y,reverse=True)
        first_max=y[0]
        second_max=y[1]
        if(second_max==0):
          continue
        ratio_array.append([int(i),first_max/second_max])

    ratio_array.sort(key=lambda x:x[1])
    
    return ratio_array


  def get_margin(self,y_predicted_margin):
    a=[]
    b=[]
    for i in range(0,y_predicted_margin.shape[0]):
        a=y_predicted_margin[i]
        c=sorted(a,reverse=True)
        difference=c[0]-c[1]
        
        b.append([int(i),difference])
    
    b.sort(key=lambda x:x[1])
    
    return b

  def get_hc(self,y_predicted_hc):
    probarray = []
    
    for i in range(0,y_predicted_hc.shape[0]):
        probarray.append([int(i),np.max(y_predicted_hc[i])])

    probarray.sort(key=lambda x:x[1],reverse=True)
    
    return probarray

  
  def plot_graph(self,epochs,accuracy,batch_size):
    fig, ax1 = plt.subplots()

    ax2 = ax1.twinx()
    ax1.plot(acc[1:],ep, 'g-')
    ax2.plot(acc[1:],b_size[1:], 'b-')

    ax1.set_xlabel('accuracy')
    ax1.set_ylabel('iterations', color='g')
    ax2.set_ylabel('sample size', color='b')

    plt.show()


  def get_iterations(self):

    checkpoint = keras.callbacks.ModelCheckpoint("AL_Model.h5", save_best_only=True, verbose=1)

    early_stopping = keras.callbacks.EarlyStopping(patience=10, verbose=1)
   
    es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

    lr_scheduler = LearningRateScheduler(lr_schedule)
    lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

    callbacks = [checkpoint, lr_reducer, lr_scheduler, es]

    if((self.type=="vgg") or (self.type=="mobilenet") or (self.type=="cnn")):
      self.model.fit(self.X_train[0:self.first], self.y_train[0:self.first], epochs=50, validation_split=0.2, callbacks=[es])
      self.model.evaluate(self.X_test,self.y_test)  

# validation_data=(self.X_train[self.first-7500:self.first],self.y_train[self.first-7500:self.first]

    emetrics=[0,0]
    
    num=0

    if(self.samplingtype=="random"):
      emetrics[1],num = self.get_iterations_rs()

      return emetrics[1],num
      


    X_train_copy=self.X_train[self.first:self.X_train.shape[0]]
    y_train_copy=self.y_train[self.first:self.X_train.shape[0]]

    batch_size=[]
    acc=[]
    ep=[]

    batch_size.append(0)
    acc.append(0)
    
    y=[]
    x=[]

    for i in range(0,self.first):
      y.append(self.y_train[i])
      x.append(self.X_train[i])
    
    while((emetrics[1]<self.goal/100)&(self.jump<=X_train_copy.shape[0])):
        
        total_index=[*range(0, X_train_copy.shape[0], 1)]
        y_predicted = self.model.predict(X_train_copy)

        if(self.samplingtype=="margin"):
          values = self.get_margin(y_predicted)
        elif(self.samplingtype=="leastconfidence"):
          values = self.get_lc(y_predicted)
        elif(self.samplingtype=="highestconfidence"):
          values = self.get_hc(y_predicted)
        elif(self.samplingtype=="entropy"):
          values = self.get_entropy(y_predicted)
        elif(self.samplingtype=="ratio"):
          values = self.get_ratio(y_predicted)
        else:
          print("Input Error.")
    
        index = []
    
        for i in range(0,self.jump):
            index.append(values[i][0])
            x.append(X_train_copy[values[i][0]])
            y.append(y_train_copy[values[i][0]])
         
        num=num+1

        # fit_generator(datagen.flow(

        if(self.type=="resnet"):
          self.model.fit(np.array(x), np.array(y), epochs=250, validation_data = (valid_X, validation_labels), callbacks=[checkpoint,early_stopping], workers=4)
            
        elif((self.type=="vgg") or (self.type=="mobilenet")):
          self.model.fit(np.array(x), np.array(y), epochs=250, validation_split=0.2, batch_size=16, callbacks=[es])

        else:
          self.model.fit(np.array(x), np.array(y), epochs=250, validation_split=0.2, callbacks=[es])

        batch_size.append(np.array(x).shape[0])
        ep.append(num)
        
        emetrics = self.model.evaluate(self.X_test, self.y_test)
        print(emetrics)
        acc.append(100*float(emetrics[1]))

        a=[]
        for element in total_index:
          if element not in index:
            a.append(element)

        X_train_copy=X_train_copy[a]
        y_train_copy=y_train_copy[a]
      
    return emetrics[1],ep,batch_size,acc


In [ ]:
s = Sampling("leastconfidence","cifar10",model,95,5000,10000,250,"resnet")

In [ ]:
e,ep,b_size,acc = s.get_iterations()

1250/1250 [==============================] - 109s 88ms/step
Epoch 1/250
469/469 [==============================] - ETA: 0s - loss: 0.3949 - accuracy: 0.8622
Epoch 1: val_loss improved from inf to 0.25055, saving model to AL_Model.h5
469/469 [==============================] - 199s 416ms/step - loss: 0.3949 - accuracy: 0.8622 - val_loss: 0.2505 - val_accuracy: 0.9236
Epoch 2/250
469/469 [==============================] - ETA: 0s - loss: 0.1733 - accuracy: 0.9374
Epoch 2: val_loss improved from 0.25055 to 0.22160, saving model to AL_Model.h5
469/469 [==============================] - 179s 382ms/step - loss: 0.1733 - accuracy: 0.9374 - val_loss: 0.2216 - val_accuracy: 0.9353
Epoch 3/250
469/469 [==============================] - ETA: 0s - loss: 0.1036 - accuracy: 0.9649
Epoch 3: val_loss did not improve from 0.22160
469/469 [==============================] - 178s 381ms/step - loss: 0.1036 - accuracy: 0.9649 - val_loss: 0.2254 - val_accuracy: 0.9356
Epoch 4/250
469/469 [====================

In [ ]:
model.save("resnet_50_best_till_now_after_sampling")

NameError: ignored

In [ ]:
!zip -r /content/resnet_50_best_till_now_after_sampling /content/resnet_50_best_till_now_after_sampling


In [ ]:
files.download("/content/AL_Model.h5")

In [ ]:
print(e)
print(ep)
print(b_size)
print(max(acc))

In [ ]:
plt.plot(ep,acc[1:])
plt.ylabel("accuracy")
plt.xlabel("iterations")

In [ ]:
plt.plot(b_size[1:],acc[1:])
plt.ylabel("accuracy")
plt.xlabel("sample size")